In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
import os
import os, sklearn, pandas, numpy as np, random
from sklearn import svm
import skimage, skimage.io, skimage.filters
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard
from sklearn.utils import shuffle
import imp

# from pcanet import PCANet

%matplotlib inline

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# set cwd back to default
os.chdir('../')
os.getcwd()

'/Users/Tristan/Downloads/dog-breed-identification/src'

In [3]:
# custom scripts
import config # params, constants
import data, models # functions that mutate outr data
# from utils import utils, plot # custom functions, in local environment

 Dataset :: namedtuple(
    ['train' = ['img_name']
    , 'test' = ['img_name']
    , 'validation' = ['img_name']
    , 'labels' = pandas.df('img_name','classification')
    , 'dict_index_to_label' = dict to convert label_index -> label_name
    , 'dict_label_to_index'= dict to convert label_name -> label_index
    


In [4]:
import data # src/data.py
dataset = data.init_dataset()

0       cardboard
1       cardboard
2       cardboard
3       cardboard
4       cardboard
5       cardboard
6       cardboard
7       cardboard
8       cardboard
9       cardboard
10      cardboard
11      cardboard
12      cardboard
13      cardboard
14      cardboard
15      cardboard
16      cardboard
17      cardboard
18      cardboard
19      cardboard
20      cardboard
21      cardboard
22      cardboard
23      cardboard
24      cardboard
25      cardboard
26      cardboard
27      cardboard
28      cardboard
29      cardboard
          ...    
2497        trash
2498        trash
2499        trash
2500        trash
2501        trash
2502        trash
2503        trash
2504        trash
2505        trash
2506        trash
2507        trash
2508        trash
2509        trash
2510        trash
2511        trash
2512        trash
2513        trash
2514        trash
2515        trash
2516        trash
2517        trash
2518        trash
2519        trash
2520        trash
2521      

### Select the amount of classes that will be used

In [5]:
# pick the n classes with the most occuring instances
amt = 5
classes = data.top_classes(dataset.labels, amt)
classes

['paper', 'glass', 'plastic', 'metal', 'cardboard']

In [6]:
def extract_topx_classes(classes, train_or_test):
    name_list = []
    n_per_class = []
    tail = '.jpg'
    for cls in classes:
        print(cls)
        names = data.items_with_label(dataset.labels, cls)
        if train_or_test == 'train':
            train_names = [f for f in names if (f) in dataset.train]
        if train_or_test == 'test':
            train_names = [f for f in names if (f) in dataset.test]
        if train_or_test == 'validation':
            train_names = [f for f in names if (f) in dataset.validation]
        name_list.append(train_names)
        n_per_class.append(len(train_names))

    n = min(n_per_class)
    # (optional) reduce n to check whether the model can rember its input
#     reduced_n = 50
#     if n > reduced_n:    n = reduced_n
    x = []
    for ls in name_list:
        for name in ls:
            x.append(name)
    random.shuffle(x)
    return x

x_train = extract_topx_classes(classes, 'train')
x_test = extract_topx_classes(classes, 'test')
x_validation = extract_topx_classes(classes, 'validation')


paper
glass
plastic
metal
cardboard
paper
glass
plastic
metal
cardboard
paper
glass
plastic
metal
cardboard


### Load and convert images

Training data

In [7]:
x_train, y_train, n = data.extract_all(dataset, x_train)
# y_test = y_train


('extract all data:', 1940)


Testing data

In [8]:
x_test, y_test, n = data.extract_all_test(dataset, x_test)


('extract all data:', 300)


Validation data

In [9]:
x_validation, y_validation, n  = data.extract_all_validation(dataset, x_validation)


('extract all data:', 150)


Convert labels and determine input and output shape for model

In [18]:
imp.reload(data)

 Dataset :: namedtuple(
    ['train' = ['img_name']
    , 'test' = ['img_name']
    , 'validation' = ['img_name']
    , 'labels' = pandas.df('img_name','classification')
    , 'dict_index_to_label' = dict to convert label_index -> label_name
    , 'dict_label_to_index'= dict to convert label_name -> label_index
    


<module 'data' from 'data.py'>

In [19]:
# y_train, y_test, y_validation = data.labels_to_vectors(dataset, y_train, y_test, y_validation)
y_train = data.one_hot(y_train)
y_test = data.one_hot(y_test)
y_validation = data.one_hot(y_validation)
input_shape = x_train.shape[1:] # = shape of an individual image (matrix)
output_length = (y_train[0]).shape[0] # = length of an individual label


Initiate VG16 network and add extra layers

In [23]:
from keras.models import Model 

def get_model(learn_rate=0.001, batches=10, dropout=0.10):
    # build the VGG16 network
    model = applications.VGG16(weights='imagenet', include_top=False, input_shape = input_shape )
    print('Model loaded.')

    # build a classifier model to put on top of the convolutional model
    # top_model = Sequential()
    # top_model.add(Flatten(input_shape=model.output_shape[1:]))
    # top_model.add(Dense(256, activation='relu'))
    # top_model.add(Dropout(0.5))
    # top_model.add(Dense(1, activation='sigmoid'))

    for layer in model.layers[0:13]:
        layer.trainable = False

    #Adding custom Layers 
    x = model.output
    x = Flatten()(x)
    x = (Dense(4096, activation='relu'))(x) #128
    x = (Dropout(dropout))(x)
    x = (Dense(4096, activation='relu'))(x) #128
    x = (Dense(16, activation='relu'))(x)
    # softmax output to get probability distribution
    predictions = Dense(output_length, activation="softmax")(x)


    # creating the final model 
    model_final = Model(input = model.input, output = predictions)

    # Adam, SGD
    # sgd = Keras.optimizers.SGD(lr=0.01, clipnorm=1.)
    optimizer = optimizers.Adam(lr=learn_rate)

    # compile the model 
    model_final.compile(loss = "categorical_crossentropy", optimizer = optimizer, metrics=['accuracy',
        'mean_squared_error','categorical_crossentropy','top_k_categorical_accuracy'])
    
    return model_final

### Hyperparameter selection

In [ ]:

from sklearn.grid_search import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import time

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

batches = [5, 10, 20]
learn_rate = [0.0001, 0.001, 0.01]
dropout = [0.1, 0.3, 0.5] # most used values
param_grid = dict(learn_rate=learn_rate, batch_size=batches, dropout=dropout)

# tune the hyperparameters via a randomized search
model = KerasClassifier(build_fn=get_model, verbose=0)
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid)
start = time.time()
grid.fit(x_validation, y_validation)
 
# evaluate the best randomized searched model on the testing
# data
print("[INFO] randomized search took {:.2f} seconds".format(
    time.time() - start))
acc = grid.score(x_validation, y_validation)
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] randomized search best parameters: {}".format(
    grid.best_params_))

data augmentation

In [21]:
#Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30)


### Train the model

In [22]:
# n epochs = n iterations over all the training data
batch_size = 5
epochs = 20

### With data augmentation

In [ ]:
model_final = get_model()

train_datagen.fit(x_train)

model_final.fit_generator(train_datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=50, epochs=epochs, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])
        
model.save(config.dataset_dir + 'models/')

Model loaded.


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


Epoch 1/20
50/50 [==============================] - 622s 12s/step - loss: 1.9029 - acc: 0.2120 - mean_squared_error: 0.1692 - categorical_crossentropy: 1.9029 - top_k_categorical_accuracy: 1.0000
Epoch 2/20
17/50 [=========>....................] - ETA: 6:40 - loss: 1.6094 - acc: 0.1529 - mean_squared_error: 0.1600 - categorical_crossentropy: 1.6094 - top_k_categorical_accuracy: 1.0000

### Without data augmentation

In [ ]:
from keras.callbacks import TensorBoard
model_final.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_split=1/6, callbacks=[TensorBoard(log_dir=config.tmp_model_dir)])

In [25]:
model_final = get_model()
model_final.summary()


Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 384, 512, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 384, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 384, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 192, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 192, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 192, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 96, 128, 128)      0      

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


#### running tests


In [ ]:
cvscores = []
scores = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

### Baseline

In [ ]:
from pcanet import PCANet

pcanet = PCANet(
        image_shape= (512,348),
        filter_shape_l1=2, step_shape_l1=1, n_l1_output=4,
        filter_shape_l2=2, step_shape_l2=1, n_l2_output=4,
        block_shape=2
    )
pcanet.validate_structure()

pcanet.fit(x_train)
X_train = pcanet.transform(x_train)
X_test = pcanet.transform(x_test)

model = RandomForestClassifier(n_estimators=100, random_state=1234, n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("accuracy: " + str(accuracy))
